#Angle based


In [1]:
import numpy as np
import pandas as pd
import os
import json

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(np.degrees(radians))
    return angle if angle <= 180 else 360 - angle

def get_avg_elbow_angle(landmarks):
    try:
        l_shoulder = landmarks['left_shoulder']
        l_elbow = landmarks['left_elbow']
        l_wrist = landmarks['left_wrist']
        left = calculate_angle(l_shoulder, l_elbow, l_wrist)

        r_shoulder = landmarks['right_shoulder']
        r_elbow = landmarks['right_elbow']
        r_wrist = landmarks['right_wrist']
        right = calculate_angle(r_shoulder, r_elbow, r_wrist)

        return (left + right) / 2
    except:
        return None

def load_json_landmarks(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    frame_landmarks = []
    for entry in data:
        if entry.get("low_confidence"):
            continue
        lm_dict = {}
        for lm in entry["landmarks"]:
            if lm["conf"] > 0.5:
                lm_dict[lm["name"]] = [lm["x"], lm["y"]]
        frame_landmarks.append({
            "frame": entry["frame"],
            "landmarks": lm_dict
        })
    return frame_landmarks

def count_bench_press_reps_from_json(json_path):
    frames = load_json_landmarks(json_path)

    stage = None
    rep_counter = 0
    rep_data = []

    entry_frame = None
    bottom_frame = None
    min_angle = 999

    for entry in frames:
        frame_num = entry["frame"]
        landmarks = entry["landmarks"]

        if not all(k in landmarks for k in ["left_shoulder", "left_elbow", "left_wrist", 
                                             "right_shoulder", "right_elbow", "right_wrist"]):
            continue

        angle = get_avg_elbow_angle(landmarks)
        if angle is None:
            continue

        if 60 < angle < 110 and stage is None:
            stage = "down"
            entry_frame = frame_num
            bottom_frame = frame_num
            min_angle = angle

        elif stage == "down":
            if angle < min_angle:
                min_angle = angle
                bottom_frame = frame_num

            if angle > 160:
                rep_counter += 1
                exit_frame = frame_num
                rep_data.append({
                    "Video_Name": os.path.basename(json_path).replace(".json", ""),
                    "Rep_Number": rep_counter,
                    "Entry_Frame": entry_frame,
                    "Bottom_Frame": bottom_frame,
                    "Exit_Frame": exit_frame
                })
                stage = None
                entry_frame = None
                bottom_frame = None
                min_angle = 999

    return rep_data

def process_json_folder(folder_path):
    all_rep_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".json"):
            path = os.path.join(folder_path, file)
            try:
                reps = count_bench_press_reps_from_json(path)
                all_rep_data.extend(reps)
                print(f"✅ {file}: {len(reps)} reps tracked")
            except Exception as e:
                print(f"❌ Error with {file}: {e}")

    df = pd.DataFrame(all_rep_data)
    csv_path = os.path.join(folder_path, "bench_press_rep_frame_tracking_from_json.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 CSV saved at: {csv_path}")
    return df

# Example usage
if __name__ == "__main__":
    folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\bench_press\Json"
    df = process_json_folder(folder)
    print("\n📊 Final Frame-Level Rep Summary:")
    print(df)


✅ bench_press_segment_1_start_02m43s729ms_pose.json: 5 reps tracked

📄 CSV saved at: C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\bench_press\Json\bench_press_rep_frame_tracking_from_json.csv

📊 Final Frame-Level Rep Summary:
                                     Video_Name  Rep_Number  Entry_Frame  \
0  bench_press_segment_1_start_02m43s729ms_pose           1           37   
1  bench_press_segment_1_start_02m43s729ms_pose           2          193   
2  bench_press_segment_1_start_02m43s729ms_pose           3          262   
3  bench_press_segment_1_start_02m43s729ms_pose           4          382   
4  bench_press_segment_1_start_02m43s729ms_pose           5          650   

   Bottom_Frame  Exit_Frame  
0            41          71  
1           224         254  
2           345         373  
3           447         585  
4           656         668  


#Peak Detection Based

In [1]:
import os
import json
import numpy as np
import pandas as pd

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(np.degrees(radians))
    return angle if angle <= 180 else 360 - angle

def json_landmarks_to_dict(landmarks):
    return {lm['name']: (lm['x'], lm['y'], lm['conf']) for lm in landmarks}

def get_avg_elbow_angle_from_dict(landmarks_dict):
    min_conf = 0.5
    required_points = ['left_shoulder', 'left_elbow', 'left_wrist', 'right_shoulder', 'right_elbow', 'right_wrist']
    for pt in required_points:
        if pt not in landmarks_dict or landmarks_dict[pt][2] < min_conf:
            return None
    l_shoulder = landmarks_dict['left_shoulder'][:2]
    l_elbow = landmarks_dict['left_elbow'][:2]
    l_wrist = landmarks_dict['left_wrist'][:2]
    left_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
    r_shoulder = landmarks_dict['right_shoulder'][:2]
    r_elbow = landmarks_dict['right_elbow'][:2]
    r_wrist = landmarks_dict['right_wrist'][:2]
    right_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
    return (left_angle + right_angle) / 2

def count_bench_press_reps_from_json_frames(frames_data, video_name):
    stage = None
    rep_counter = 0
    rep_data = []

    entry_frame = None
    bottom_frame = None
    min_angle = 999

    for frame_info in frames_data:
        frame_num = frame_info.get('frame', None)
        if frame_num is None:
            continue
        landmarks_dict = json_landmarks_to_dict(frame_info['landmarks'])
        angle = get_avg_elbow_angle_from_dict(landmarks_dict)
        if angle is None:
            continue

        if 60 < angle < 110 and stage is None:
            stage = "down"
            entry_frame = frame_num
            min_angle = angle
            bottom_frame = frame_num

        elif stage == "down":
            if angle < min_angle:
                min_angle = angle
                bottom_frame = frame_num

            if angle > 160:
                rep_counter += 1
                exit_frame = frame_num
                rep_data.append({
                    "Video_Name": video_name,
                    "Rep_Number": rep_counter,
                    "Entry_Frame": entry_frame,
                    "Bottom_Frame": bottom_frame,
                    "Exit_Frame": exit_frame
                })
                stage = None
                entry_frame = None
                bottom_frame = None
                min_angle = 999

    return rep_data

def process_json_folder(folder_path):
    all_rep_data = []
    valid_exts = ('.json',)

    for file in os.listdir(folder_path):
        if file.lower().endswith(valid_exts):
            full_path = os.path.join(folder_path, file)
            try:
                with open(full_path, 'r') as f:
                    frames_data = json.load(f)
                reps = count_bench_press_reps_from_json_frames(frames_data, file)
                all_rep_data.extend(reps)
                print(f"✅ {file}: {len(reps)} reps tracked")
            except Exception as e:
                print(f"❌ Error processing {file}: {e}")

    df = pd.DataFrame(all_rep_data)
    csv_path = os.path.join(folder_path, "bench_press_rep_frame_tracking_summary.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 CSV saved at: {csv_path}")
    return df

# Example usage:
if __name__ == "__main__":
    folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\bench_press\Json"
    df = process_json_folder(folder)
    print("\n📊 Final Frame-Level Rep Summary:")
    print(df)


✅ bench_press_segment_1_start_02m43s729ms_pose.json: 5 reps tracked

📄 CSV saved at: C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\bench_press\Json\bench_press_rep_frame_tracking_summary.csv

📊 Final Frame-Level Rep Summary:
                                          Video_Name  Rep_Number  Entry_Frame  \
0  bench_press_segment_1_start_02m43s729ms_pose.json           1           37   
1  bench_press_segment_1_start_02m43s729ms_pose.json           2          193   
2  bench_press_segment_1_start_02m43s729ms_pose.json           3          262   
3  bench_press_segment_1_start_02m43s729ms_pose.json           4          382   
4  bench_press_segment_1_start_02m43s729ms_pose.json           5          650   

   Bottom_Frame  Exit_Frame  
0            41          71  
1           224         254  
2           345         373  
3           447         585  
4           656         668  
